In [1]:
! pip install datasets evaluate transformers transformers[torch] rouge-score nltk --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [2]:
import transformers
from datasets import load_dataset, concatenate_datasets
from evaluate import load
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline
from huggingface_hub import notebook_login
import numpy as np
import nltk

In [3]:
notebook_login()

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
torch.cuda.set_device(device)
torch.cuda.current_device()

0

In [ ]:
#datasets_tr = load_dataset("MoritzLaurer/multilingual-NLI-26lang-2mil7", split=["tr_anli", "tr_fever", "tr_ling", "tr_mnli", "tr_wanli"])

In [7]:
dataset_files = {"tr_anli": "tr_anli.parquet",
                 "tr_fever": "tr_fever.parquet",
                 "tr_ling": "tr_ling.parquet",
                 "tr_mnli": "tr_mnli.parquet",
                 "tr_wanli": "tr_wanli.parquet"}
dict_tr = load_dataset("parquet", data_files=dataset_files)
datasets_tr = [dict_tr['tr_anli'], dict_tr['tr_fever'], dict_tr['tr_ling'], dict_tr['tr_mnli'], dict_tr['tr_wanli']]

Generating tr_anli split: 0 examples [00:00, ? examples/s]

Generating tr_fever split: 0 examples [00:00, ? examples/s]

Generating tr_ling split: 0 examples [00:00, ? examples/s]

Generating tr_mnli split: 0 examples [00:00, ? examples/s]

Generating tr_wanli split: 0 examples [00:00, ? examples/s]

In [16]:
dataset = concatenate_datasets(datasets_tr)
dataset = dataset.remove_columns(["premise_original", "hypothesis_original"])

In [17]:
dataset

Dataset({
    features: ['label', 'premise', 'hypothesis'],
    num_rows: 105000
})

In [18]:
label_col = ['gereklilik' if x[1] == 0 else 'nötr' if x[1] == 1 else 'çelişki' for x in enumerate(dataset['label'])]

In [19]:
dataset = dataset.rename_column("label", "labelnum")
dataset = dataset.add_column("labelstr", label_col)

In [20]:
dataset_shuffled = dataset.shuffle(seed=42)
dataset_eval = dataset_shuffled.select(range(5000))
dataset_shuffled = dataset_shuffled.select(range(5000, len(dataset_shuffled)))
dataset_ranged = dataset_shuffled.select(range(100000))

In [21]:
dataset_splitted = dataset_ranged.train_test_split(test_size=0.2, seed=42)

In [22]:
print(dataset_splitted['train'].shape, dataset_splitted['test'].shape)

(80000, 4) (20000, 4)


In [23]:
dataset_splitted['train'][0]

{'labelnum': 0,
 'premise': "1983'te Kıbrıs Türkleri, Kuzey Kıbrıs Türk Cumhuriyeti (KKTC) adını alarak ayrı bir devlet ilan ettiler. Barış görüşmeleri ara sıra yapıldı, ancak Kıbrıs bölünmüş kaldı.",
 'hypothesis': 'Barış görüşmeleri devletin birleşmesine yardımcı olmadı.',
 'labelstr': 'gereklilik'}

In [24]:
tokenizer = AutoTokenizer.from_pretrained("boun-tabi-LMG/turna_nli_nli_tr")

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.68k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
def prompt_format(row):
  return f"hipotez: {row[1]} önerme: {row[0]}"

In [26]:
max_input_length = 512
max_target_length = 1

def preprocess_function(examples):
  inputs = [prompt_format(x) for x in zip(examples['premise'], examples['hypothesis'])]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
  labels = tokenizer(text_target=examples["labelstr"], max_length=max_target_length, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [27]:
dataset_encoded = dataset_splitted.map(preprocess_function, batched=True)

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [28]:
model = AutoModelForSeq2SeqLM.from_pretrained("boun-tabi-LMG/turna_nli_nli_tr")

config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [29]:
metric = load("rouge")

In [30]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    result = {key: value * 100 for key, value in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

In [31]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [32]:
args = Seq2SeqTrainingArguments(
    "turna-nlitr-finetuned-nli_",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    predict_with_generate=True,
    push_to_hub=True,
)

In [33]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [34]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.782300,0.755696,36.955000,36.955000,36.960000,36.955000,2.446200


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=20000, training_loss=0.7907528915405273, metrics={'train_runtime': 16809.5375, 'train_samples_per_second': 4.759, 'train_steps_per_second': 1.19, 'total_flos': 2.7252926447616e+17, 'train_loss': 0.7907528915405273, 'epoch': 1.0})

In [35]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/aniltepe/turna-nlitr-finetuned-nli_/commit/206f2163c2a8d41f61a3cf9a95eb7d4373862c75', commit_message='End of training', commit_description='', oid='206f2163c2a8d41f61a3cf9a95eb7d4373862c75', pr_url=None, pr_revision=None, pr_num=None)